In [1]:
import copy

In [2]:
Int_Tree = [[0.03989], 
           [0.04704, 0.03969], 
           [0.05497, 0.04639, 0.03915], 
           [0.06375, 0.05380, 0.04540, 0.03832], 
           [0.07345, 0.06199, 0.05231, 0.04415, 0.03762], 
           [0.08413, 0.07100, 0.05992, 0.05056, 0.04267, 0.03601]
          ]
Amount_ = 100000
InitRate_ = 0.04640
Period_ = 6

In [3]:
def OptionPrice(IntTree, Amount, InitRate, Period):
    """
    Calculate mortgage option prices using a binomial tree approach.
    
    Parameters:
    - IntTree: Interest rate tree structure
    - Amount: Initial mortgage amount
    - InitRate: Initial interest rate
    - Period: Number of periods
    """
    # Calculate initial payment and principal
    rate = InitRate / 2
    pmt = (Amount * rate) / (1 - (1 + rate) ** (-Period))
    first_left_principal = Amount - (pmt - rate * Amount)
    
    # 1. Calculate Beginning Balance of Principals for each period
    principal_list = [Amount, first_left_principal]
    for i in range(2, Period + 1):
        principal_deduction = principal_list[i-1] * rate
        principal_list.append(principal_list[i-1] - (pmt - principal_deduction))
        
    print('BB of Principle: \n')    
    for i, principal in enumerate(principal_list):    
        print(f'Year {i+1}: {round(principal, 2)}\n')
        
    print('-------------------Separate Line-------------------\n')
        
    # 2. Calculate Mortgage value without prepayment option
    mortgage_market_value = []
    final_values = [0] * (Period + 1)
    mortgage_market_value.append(final_values)

    for year in range(Period, 0, -1):
        year_market_values = []
        
        for j in range(len(mortgage_market_value[-1]) - 1):
            # Calculate present value using next year's values
            pv_next_year = (
                0.5 * mortgage_market_value[-1][j] + 
                0.5 * mortgage_market_value[-1][j + 1]
            )
            
            current_rate = IntTree[year - 1][j]
            present_value = (pmt + pv_next_year) / (1 + current_rate / 2)
            year_market_values.append(present_value)

        mortgage_market_value.append(year_market_values)

    mortgage_market_value.reverse()
    print('Mortgage Market Value: \n')
    for year, values in enumerate(mortgage_market_value):
        print(f'Year {year+1} : {values}\n')
    
    print('-------------------Separate Line-------------------\n')
    
    # 3. Calculate Prepayment Option Price
    option_price = []
    inverted_mortgage_value = mortgage_market_value[::-1][1:]  # Remove first element
    option_price.append([0] * (Period + 1))
    
    for year_index, year_values in enumerate(inverted_mortgage_value):
        yearly_option_prices = []
        principal_index = len(year_values) - 1
        
        for node_index in range(len(year_values)):
            if node_index + 1 >= len(option_price[year_index]):
                break
                
            current_rate = IntTree[principal_index][node_index]
            future_option_value = (option_price[year_index][node_index] + 
                                 option_price[year_index][node_index + 1]) / 2
            
            if year_values[node_index] < principal_list[principal_index]:
                result = max(0, future_option_value) / (1 + current_rate / 2)
            else:
                result = max(
                    (year_values[node_index] - principal_list[principal_index]),
                    future_option_value
                ) / (1 + current_rate / 2)
                
            yearly_option_prices.append(result)
                
        option_price.append(yearly_option_prices)
        
    option_price.reverse()
    print('Option Value: \n')
    for year, values in enumerate(option_price):
        print(f'Year {year+1} : {values}\n')   
    
    print('-------------------Separate Line-------------------\n')
    
    # 4. Calculate Mortgage Value to Seller With Prepayment Option
    mortgage_with_prepaid = copy.deepcopy(mortgage_market_value)
    
    for year in range(len(mortgage_market_value)):
        for i, (market_value, option_value) in enumerate(
            zip(mortgage_with_prepaid[year], option_price[year])
        ):
            if (market_value - option_value) < principal_list[year]:
                mortgage_with_prepaid[year][i] = market_value - option_value
            else:
                mortgage_with_prepaid[year][i] = principal_list[year]
    
    print('Mortgage w/ Prepaid Opt $ to Seller: \n')
    for year, values in enumerate(mortgage_with_prepaid):
        print(f'Yr {year} : {values}\n')

OptionPrice(Int_Tree, Amount_, InitRate_, Period_)




BB of Principle: 

Year 1: 100000

Year 2: 84274.14

Year 3: 68183.45

Year 4: 51719.44

Year 5: 34873.48

Year 6: 17636.69

Year 7: 0.0

-------------------Separate Line-------------------

Mortgage Market Value: 

Year 1 : [100174.32539140608]

Year 2 : [83638.37195898221, 84614.51833448966]

Year 3 : [67188.91319354619, 67930.46541216527, 68565.20716030648]

Year 4 : [50736.421232167646, 51243.06538397047, 51677.44540162693, 52045.582450108515]

Year 5 : [34163.49894474625, 34452.07604393625, 34699.217312458786, 34910.11518282543, 35083.72210767625]

Year 6 : [17317.400703988475, 17427.19185379213, 17520.93009884049, 17600.906254488287, 17668.891367280812, 17726.688144558968]

Year 7 : [0, 0, 0, 0, 0, 0, 0]

-------------------Separate Line-------------------

Option Value: 

Year 1 : [170.91646256031152]

Year 2 : [4.183504671109546, 333.7521869415614]

Year 3 : [0.0, 8.563801401948085, 374.43187183752764]

Year 4 : [0.0, 0.0, 17.524877550932544, 320.00656219859775]

Year 5 : [0.0,